### **Intalación dependencias**

In [1]:
pip install --upgrade "osmnx>=2.0.0" "geopandas>=1.0" "shapely>=2.0" "pyproj>=3.5" "rtree" "requests"

Note: you may need to restart the kernel to use updated packages.


### **Funciones auxiliares**

In [2]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point, LineString
import folium
import json
from shapely.ops import unary_union
import re


PLACE = "Rome, Italy"

ox.settings.log_console = False
ox.settings.use_cache = True


def invert_coords(coords):
    return [[c[1], c[0]] for c in coords]

def buffer_points(data, radius_m=60):
    if data.empty:
        return gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")
    g = data.to_crs(3857)
    g["geometry"] = g.buffer(radius_m)
    return g.to_crs(4326)

### **1.- Áreas arqueológicas** *(rojo)*

La instalación de estaciones de carga de patinetes queda totalmente prohibida, porque son bienes culturales protegidos *(Codice dei Beni Culturali)*.

- Protegidas por ley bajo el D.Lgs. 42/2004
    - Prohibido instalar elementos fijos o mobiliario urbano sin autorización de:

        - Soprintendenza Archeologica di Roma
        - Ministero della Cultura (MiC)
    - Riesgo de impacto físico en zonas frágiles
    - Flujos turísticos muy elevados

- Piano Regolatore Generale di Roma – Zone “A” (centro histórico)

    - Las áreas arqueológicas están sujetas a vincolo archeologico totale.
    - No se permite introducir elementos urbanos nuevos (como estaciones de movilidad).

Las APIs de OSM, nos ofrecen una serie de tags que permiten seleccionar monumentos de forma "automática", pero por una cuestión de versiones, lo prepararemos para que cuando el "modo automático" no funcione, prepararemos nosotros mismo la bbox.

Por estos motivos restringiremos:
- Foro Romano
- Fori Imperiali
- Palatino
- Domus Aurea
- Mercati di Traiano
- Largo Argentina
- Circus Maximus
- Colosseo

In [3]:
'''from shapely.ops import unary_union

def build_archaeo_zone(name_keywords, buffer_m=10):
    tags_list = [
        {"historic": "archaeological_site"},
        {"historic": "ruins"},
        {"tourism": "attraction"},
        {"boundary": "protected_area"},
        {"leisure": "park"},
        {"landuse": "recreation_ground"},
        {"area": "yes"},
    ]

    collected = []

    for tags in tags_list:
        try:
            gdf = ox.features_from_place(PLACE, tags=tags)
            gdf = gdf[gdf.geom_type.isin(["Polygon", "MultiPolygon"])]
            gdf = gdf[gdf["name"].str.contains("|".join(name_keywords), na=False)]
            if len(gdf) > 0:
                collected.append(gdf)
        except:
            pass

    if len(collected) == 0:
        print(f"No se encontró nada para {name_keywords}")
        return None

    merged = pd.concat(collected, ignore_index=True)
    geom = unary_union(merged.geometry)

    # suavizado doble
    g = gpd.GeoSeries([geom], crs="EPSG:4326").to_crs(3857)
    g = g.buffer(buffer_m).buffer(-buffer_m / 2)
    g = g.to_crs(4326)

    print(f"Zona arqueológica reconstruida: {name_keywords}")
    return gpd.GeoDataFrame(geometry=g)


# ================================================================
# 8 ÁREAS ARQUEOLÓGICAS 
# ================================================================

FORO_ROMANO      = build_archaeo_zone(["Foro Romano", "Roman Forum"])
FORI_IMPERIALI   = build_archaeo_zone(["Fori Imperiali", "Imperial Fora"])
PALATINO         = build_archaeo_zone(["Palatino"])
DOMUS_AUREA      = build_archaeo_zone(["Domus Aurea"])
CIRCUS_MAXIMUS   = build_archaeo_zone(["Circo Massimo", "Circus Maximus"])
LARGO_ARGENTINA  = build_archaeo_zone(["Largo di Torre Argentina", "Argentina"])
TRAJANO          = build_archaeo_zone(["Mercati di Traiano", "Traiano"])
COLOSSEO         = build_archaeo_zone(["Colosseo", "Colosseum"])


# ================================================================
# MERGE FINAL PARA USAR LUEGO EN TU PIPELINE
# ================================================================
archaeo_final = gpd.GeoDataFrame(
    pd.concat([
        z for z in [
            FORO_ROMANO,
            FORI_IMPERIALI,
            PALATINO,
            DOMUS_AUREA,
            CIRCUS_MAXIMUS,
            LARGO_ARGENTINA,
            TRAJANO,
            COLOSSEO
        ] if z is not None
    ], ignore_index=True),
    crs="EPSG:4326"
)

print("Total zonas arqueológicas generadas:", len(archaeo_final))'''

'from shapely.ops import unary_union\n\ndef build_archaeo_zone(name_keywords, buffer_m=10):\n    tags_list = [\n        {"historic": "archaeological_site"},\n        {"historic": "ruins"},\n        {"tourism": "attraction"},\n        {"boundary": "protected_area"},\n        {"leisure": "park"},\n        {"landuse": "recreation_ground"},\n        {"area": "yes"},\n    ]\n\n    collected = []\n\n    for tags in tags_list:\n        try:\n            gdf = ox.features_from_place(PLACE, tags=tags)\n            gdf = gdf[gdf.geom_type.isin(["Polygon", "MultiPolygon"])]\n            gdf = gdf[gdf["name"].str.contains("|".join(name_keywords), na=False)]\n            if len(gdf) > 0:\n                collected.append(gdf)\n        except:\n            pass\n\n    if len(collected) == 0:\n        print(f"No se encontró nada para {name_keywords}")\n        return None\n\n    merged = pd.concat(collected, ignore_index=True)\n    geom = unary_union(merged.geometry)\n\n    # suavizado doble\n    g =

Polígonos que fallaron a la hora de detactarlos, y dibujamos manualmente:

In [4]:
PLACE = "Rome, Italy"

manual_polygons = {
    "Foro Romano": Polygon([
        (12.4847, 41.8923),
        (12.4856, 41.8932),
        (12.4870, 41.8935),
        (12.4883, 41.8928),
        (12.4881, 41.8919),
        (12.4870, 41.8913),
        (12.4858, 41.8915),
    ]),

    "Fori Imperiali": Polygon([
        (12.4828, 41.8941),
        (12.4844, 41.8953),
        (12.4870, 41.8950),
        (12.4876, 41.8941),
        (12.4870, 41.8934),
        (12.4850, 41.8936),
    ]),

    "Domus Aurea": Polygon([
        (12.4942, 41.8918),
        (12.4950, 41.8924),
        (12.4963, 41.8923),
        (12.4966, 41.8916),
        (12.4957, 41.8911),
    ]),

    "Circo Massimo": Polygon([
        (12.4848, 41.8871),
        (12.4887, 41.8871),
        (12.4894, 41.8885),
        (12.4871, 41.8896),
        (12.4850, 41.8890),
    ])
}

Uso de OSM, si falla, se realizarán manualmente. (3 áreas definidas anteriormente)

In [5]:
def build_archaeo_zone(name_keywords, buffer_m=10):
    main_name = name_keywords[0]

    tags_list = [
        {"historic": "archaeological_site"},
        {"historic": "ruins"},
        {"tourism": "attraction"},
        {"boundary": "protected_area"},
        {"leisure": "park"},
        {"landuse": "recreation_ground"},
        {"area": "yes"},
    ]

    collected = []

    for tags in tags_list:
        try:
            gdf = ox.features_from_place(PLACE, tags=tags)
            gdf = gdf[gdf.geom_type.isin(["Polygon", "MultiPolygon"])]
            gdf = gdf[gdf["name"].str.contains("|".join(name_keywords), na=False)]
            if len(gdf) > 0:
                collected.append(gdf)
        except:
            pass

    if len(collected) == 0:
        if main_name in manual_polygons:
            print(f"OSM no encontró {main_name} → usando polígono manual.")
            return gpd.GeoDataFrame(
                geometry=[manual_polygons[main_name]], crs="EPSG:4326"
            )
        else:
            print(f"No se encontró nada para {name_keywords} y no hay polígono manual.")
            return None

    merged = pd.concat(collected, ignore_index=True)
    geom = unary_union(merged.geometry)

    # suavizado
    g = gpd.GeoSeries([geom], crs="EPSG:4326").to_crs(3857)
    g = g.buffer(buffer_m).buffer(-buffer_m / 2)
    g = g.to_crs(4326)

    print(f"Zona arqueológica reconstruida: {name_keywords}")
    return gpd.GeoDataFrame(geometry=g)

Dibujamos las 8 zonas arqueológicas

In [6]:
FORO_ROMANO      = build_archaeo_zone(["Foro Romano", "Roman Forum"])
FORI_IMPERIALI   = build_archaeo_zone(["Fori Imperiali", "Imperial Fora"])
PALATINO         = build_archaeo_zone(["Palatino"])
DOMUS_AUREA      = build_archaeo_zone(["Domus Aurea"])
CIRCUS_MAXIMUS   = build_archaeo_zone(["Circo Massimo", "Circus Maximus"])
LARGO_ARGENTINA  = build_archaeo_zone(["Largo di Torre Argentina", "Argentina"])
TRAJANO          = build_archaeo_zone(["Mercati di Traiano", "Traiano"])
COLOSSEO         = build_archaeo_zone(["Colosseo", "Colosseum"])

OSM no encontró Foro Romano → usando polígono manual.
OSM no encontró Fori Imperiali → usando polígono manual.
Zona arqueológica reconstruida: ['Palatino']
OSM no encontró Domus Aurea → usando polígono manual.
Zona arqueológica reconstruida: ['Circo Massimo', 'Circus Maximus']
Zona arqueológica reconstruida: ['Largo di Torre Argentina', 'Argentina']
Zona arqueológica reconstruida: ['Mercati di Traiano', 'Traiano']
Zona arqueológica reconstruida: ['Colosseo', 'Colosseum']


Realizamos el merge final

In [7]:
archaeo_final = gpd.GeoDataFrame(
    pd.concat(
        [z for z in [
            FORO_ROMANO,
            FORI_IMPERIALI,
            PALATINO,
            DOMUS_AUREA,
            CIRCUS_MAXIMUS,
            LARGO_ARGENTINA,
            TRAJANO,
            COLOSSEO
        ] if z is not None],
        ignore_index=True
    ),
    crs="EPSG:4326"
)

print("Total zonas arqueológicas generadas:", len(archaeo_final))

Total zonas arqueológicas generadas: 8


### **2.- Plazas Históricas** *(azul)* 

Prohibido el estacionamiento por ordenanza municial. Sólo pueden transitar personas en estas plazas. (La mayoría 100% peatonales)

La UNESCO las declaró Patrimonio de la Humanidad.
Son lugares con enorme afluencia turística, por lo que podría generar:
- congestión 
- riesgo de accidentes
- mala imagen del servicio

Al igual que en el anterior caso, intentaremos seleccionarlos de forma automática, y si no detecta las etiquetas hacerlo manualmente.

Las plazas seleccionadas son:
- Piazza Navona
- Piazza di Spagna
- Piazza Venezia (parte central)
- Piazza del Popolo
- Piazza della Rotonda (Pantheon)
- Piazza Campo de’ Fiori
- Piazza Barberini (zona de la fuente)
- Piazza Trilussa (Trastevere)

In [8]:
plazas = ox.features_from_place(PLACE, {"place": "square"})

important_plazas = [
    "Piazza Navona",
    "Piazza di Spagna",
    "Piazza del Popolo",
    "Piazza della Rotonda",
    "Piazza Barberini",
    "Piazza Trilussa",
    "Piazza Venezia"
]

plaza_polygons = plazas[
    plazas.geom_type.isin(["Polygon", "MultiPolygon"]) &
    plazas["name"].isin(important_plazas)
]

campo = ox.features_from_place(PLACE, {"amenity": "marketplace"})
campo_polygons = campo[
    campo.geom_type.isin(["Polygon", "MultiPolygon"]) &
    campo["name"].eq("Campo de' Fiori")
]

plaza_final = pd.concat(
    [plaza_polygons, campo_polygons],
    ignore_index=True
)

### **3.- Monumentos Turísticos** *(amarillo)*

El estacionamiento de cualquier vehículo está prohibido por la ordenanza municipal de Roma. Además de que no pondremos estaciones ya que son zonas de altísima concentración turística.


El entorno se considera  “área monumental”, protegido por:

- Codice dei Beni Culturali
- Decreto Legislativo sobre monumentos históricos

Los monumentos seleccionados serán: 
- Panteón
- Fontana di Trevi
- Castel Sant’Angelo
- Altare della Patria
- Bocca della Verità
- Galleria Borghese

In [9]:
monument_keywords = {
    "Pantheon": ["Pantheon", "Panteon", "Tempio della Rotonda"],
    "Fontana di Trevi": ["Fontana di Trevi", "Trevi Fountain"],
    "Castel Sant'Angelo": ["Castel Sant'Angelo", "Mausoleo di Adriano"],
    "Altare della Patria": ["Altare della Patria", "Vittoriano", "Monumento Nazionale a Vittorio Emanuele II"],
    "Bocca della Verità": ["Bocca della Verità", "Bocca della Verita"],
    "Galleria Borghese": ["Galleria Borghese", "Museo Borghese"]
}

# Tags OSM más amplios para asegurar detección
monument_tags = {
    "tourism": ["attraction", "museum", "artwork", "monument"],
    "historic": ["monument"],
    "amenity": ["fountain"]
}

tourism_raw = ox.features_from_place(
    PLACE,
    tags=monument_tags
)

all_monument_geoms = []

for monument, keywords in monument_keywords.items():

    # Buscar polígonos OSM
    poly = tourism_raw[
        tourism_raw.geom_type.isin(["Polygon", "MultiPolygon"]) &
        tourism_raw["name"].str.contains("|".join(keywords), na=False)
    ]

    # Buscar puntos OSM
    pts = tourism_raw[
        (tourism_raw.geom_type == "Point") &
        tourism_raw["name"].str.contains("|".join(keywords), na=False)
    ]

    pts_buffered = buffer_points(pts, radius_m=50)

    manual_poly = None

    if len(poly) == 0 and len(pts_buffered) == 0:

        if monument == "Pantheon":
            manual_poly = Point(12.4769, 41.8986).buffer(40)

        elif monument == "Fontana di Trevi":
            manual_poly = Point(12.4833, 41.9009).buffer(35)

        elif monument == "Castel Sant'Angelo":
            manual_poly = Point(12.4663, 41.9031).buffer(70)

        elif monument == "Altare della Patria":
            manual_poly = Point(12.4828, 41.8956).buffer(60)

        elif monument == "Bocca della Verità":
            manual_poly = Point(12.4823, 41.8880).buffer(25)

        elif monument == "Galleria Borghese":
            manual_poly = Point(12.4922, 41.9142).buffer(80)

        if manual_poly is not None:
            manual_gdf = gpd.GeoDataFrame(geometry=[manual_poly], crs="EPSG:4326")
            all_monument_geoms.append(manual_gdf)
            print(f"Usando polígono manual para: {monument}")
            continue

    if len(poly) > 0:
        all_monument_geoms.append(poly[["geometry"]])
        print(f"Polígono OSM encontrado: {monument}")

    if len(pts_buffered) > 0:
        all_monument_geoms.append(pts_buffered[["geometry"]])
        print(f"Punto bufferizado para: {monument}")

monuments_final = gpd.GeoDataFrame(
    pd.concat(all_monument_geoms, ignore_index=True),
    crs="EPSG:4326"
)

Polígono OSM encontrado: Pantheon
Polígono OSM encontrado: Fontana di Trevi
Usando polígono manual para: Castel Sant'Angelo
Polígono OSM encontrado: Altare della Patria
Punto bufferizado para: Bocca della Verità
Punto bufferizado para: Galleria Borghese


### **4.- Vías peatonales importantes** *(verde)*

Son calles donde no se permite el acceso con vehículos personales, por lo que estimamos que tampoco debemos colocar estaciones. No por una cuestión legal, sino por no entorpecer el paso de peatones. Muchas de estas calles son largas y albergan tiendas, por lo que aunque exista mucha afluencia de patinetes, es más óptimo ponerlas en una calle paralela (p.e)

Prioridad peatonal según el Comune di Roma. Por lo que las estaciones entoorpecerían el flujo de personas.

Las vías seleccionadas son:
- Via dei Fori Imperiali (casi toda)
- Via Margutta
- Via dei Coronari
- Via della Conciliazione (parte turística)
- Corso Vittorio Emanuele (algunos tramos peatonales)

In [10]:
important_streets_dict = {
    "Via dei Fori Imperiali": {
        "keywords": ["Fori Imperiali"],
        "buffer": 35
    },
    "Via Margutta": {
        "keywords": ["Margutta"],
        "buffer": 12
    },
    "Via dei Coronari": {
        "keywords": ["Coronari"],
        "buffer": 15
    },
    "Via della Conciliazione": {
        "keywords": ["Conciliazione"],
        "buffer": 25
    },
    "Corso Vittorio Emanuele": {
        "keywords": ["Corso Vittorio", "Vittorio Emanuele II"],
        "buffer": 20
    },
}

street_tags = {
    "highway": [
        "pedestrian",
        "footway",
        "living_street",
        "path",
        "service"
    ]
}

streets_osm = ox.features_from_place(PLACE, tags=street_tags)

ways_by_name = {}

for street_name, info in important_streets_dict.items():
    keywords = info["keywords"]

    regex = "|".join([re.escape(k) for k in keywords])
    match = streets_osm[
        (streets_osm.geom_type == "LineString") &
        (streets_osm["name"].str.contains(regex, na=False, case=False))
    ]

    if len(match) > 0:
        print(f"Encontrada en OSM: {street_name}")
        ways_by_name[street_name] = match.copy()
    else:
        print(f"No encontrada, reconstruyendo: {street_name}")

        if street_name == "Via dei Fori Imperiali":
            coords = [
                (12.4847, 41.8933),
                (12.4870, 41.8922)
            ]
        elif street_name == "Via della Conciliazione":
            coords = [
                (12.4574, 41.9023),
                (12.4616, 41.9023)
            ]
        elif street_name == "Corso Vittorio Emanuele":
            coords = [
                (12.4681, 41.8983),
                (12.4700, 41.8987),
                (12.4740, 41.8986),
                (12.4770, 41.8977)
            ]
        else:
            coords = []

        ways_by_name[street_name] = gpd.GeoDataFrame(
            geometry=[LineString(coords)],
            crs="EPSG:4326"
        )

buffered_ways = []

for street_name, info in important_streets_dict.items():
    buf = info["buffer"]

    gdf = ways_by_name[street_name].to_crs(3857)
    gdf["geometry"] = gdf.buffer(buf)
    buffered_ways.append(gdf.to_crs(4326))

ways_final = pd.concat(buffered_ways, ignore_index=True)

Encontrada en OSM: Via dei Fori Imperiali
Encontrada en OSM: Via Margutta
Encontrada en OSM: Via dei Coronari
Encontrada en OSM: Via della Conciliazione
Encontrada en OSM: Corso Vittorio Emanuele


### **5.- Ciudad del Vaticano**

La ciudad del Vaticano es un estado soberano independiente

- No pertenece al municipio de Roma.
- Zona con protección militar y religiosa.
- No se pueden instalar estaciones de ningún servicio dentro del Vaticano sin permiso estatal.

In [11]:
ox.settings.log_console = False
ox.settings.use_cache = True

def load_vatican():
    try:
        vdf = ox.geocode_to_gdf("Vatican City")
        print("Vaticano cargado correctamente")
        return vdf[["geometry"]]
    except Exception as e:
        print("Error cargando Vaticano:", e)
        return gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

VATICANO = load_vatican()

Vaticano cargado correctamente


### **6.- JSON Final**

In [12]:
combined = gpd.GeoDataFrame(
    pd.concat([
        archaeo_final[["geometry"]],
        plaza_final[["geometry"]],
        monuments_final[["geometry"]],
        ways_final[["geometry"]],
    ], ignore_index=True),
    crs="EPSG:4326"
)

restricted_json = {"restricted_polygons": []}

for geom in combined.geometry:
    if geom.geom_type == "Polygon":
        restricted_json["restricted_polygons"].append(
            list(map(list, geom.exterior.coords))
        )
    elif geom.geom_type == "MultiPolygon":
        for poly in geom.geoms:
            restricted_json["restricted_polygons"].append(
                list(map(list, poly.exterior.coords))
            )

with open("restricted_zones.json", "w") as f:
    json.dump(restricted_json, f, indent=2)

print("restricted_zones.json generado correctamente.")

restricted_zones.json generado correctamente.


### **7.- Filtración área gigante (temporal)**

Tuvimos un problema con uno de los monumentos que lo cogía de forma gigante, y ocupaba media Europa.

In [13]:
def remove_large(gdf, max_area=2_000_000):  # 2 km2
    if gdf.empty:
        return gdf
    g = gdf.to_crs(3857)
    g["area"] = g.area
    g = g[g["area"] < max_area]
    return g.to_crs(4326)

archaeo_final = remove_large(archaeo_final)
plaza_final = remove_large(plaza_final)
monuments_final = remove_large(monuments_final)
ways_final = remove_large(ways_final)

c:\Users\Lenovo\anaconda3\Lib\site-packages\shapely\measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)


### **8.- Mapa Final HTML**

In [14]:
m = folium.Map(location=[41.896, 12.482], zoom_start=13)

def add_layer(gdf, color, name):
    fg = folium.FeatureGroup(name=name)
    for geom in gdf.geometry:
        if geom.geom_type == "Polygon":
            folium.Polygon(
                locations=invert_coords(geom.exterior.coords),
                color=color, fill=True, weight=1, fill_opacity=0.35
            ).add_to(fg)
        elif geom.geom_type == "MultiPolygon":
            for poly in geom.geoms:
                folium.Polygon(
                    locations=invert_coords(poly.exterior.coords),
                    color=color, fill=True, weight=1, fill_opacity=0.35
                ).add_to(fg)
    fg.add_to(m)

add_layer(archaeo_final, "red", "Áreas arqueológicas")
add_layer(plaza_final, "blue", "Plazas peatonales")
add_layer(monuments_final, "yellow", "Monumentos turísticos")
add_layer(ways_final, "green", "Vías peatonales importantes")

#En vez de hacerlo igual, comprobamos que se crea correctamente por unos problemas que daba
if VATICANO is not None and len(VATICANO) > 0:
    add_layer(VATICANO, "purple", "Città del Vaticano")
else:
    print("Vaticano no disponible")

folium.LayerControl().add_to(m)
m.save("restricted_map.html")

print("restricted_map.html generado correctamente.")

restricted_map.html generado correctamente.
